# Enunciado


1. Pretende-se um protótipo protocolo $\,{N\choose{N-1}}\,$-OT, usando a abordagem $\,\mathsf{LPN}\,$ “Learning Parity with Noise” +Capítulo 6d:  Oblivious Linear Evaluation para mensagens de $\,n\,$ bytes (i.e. $\,8\times n\,$bits) que possa ser convertido para mensagens $\,m\in \mathbb{F}_p^n\,$ (vetores de $\,n\,$componentes no corpo finito  $\,\mathbb{F}_p\,$). Para isso
    1. Implemente um protótipo do protocolo $\,\mathsf{LPN}$ $\,{N\choose{N-1}}$-OT  para mensagens de $\,n\,$ bytes (i.e. $\,8\times n\,$bits). Ver +Capítulo 6d:  Oblivious Linear Evaluation .
    2. Codificando os elementos de um corpo primo $\;\mathbb{F}_p\;$ em “arrays” de “bytes” , converta a solução anterior num protocolo $\,{N\choose{N-1}}$-OT em que as mensagens são  vetores $\,\mathbb{F}_p^\ell\,$.

----------

## Definição de variáveis

In [1]:
p = 7  
F = GF(p)
n = 8
V = VectorSpace(F, n)  # Espaço vetorial F_p^n
V

Vector space of dimension 8 over Finite Field of size 7

In [2]:
m1 = V([1, 3, 5, 6, 2, 0, 4, 1])  # Exemplo de vetor em F_p^n
print(m1)
m2 = V.random_element()  # Gera um vetor aleatório em F_p^n
print(m2)

(1, 3, 5, 6, 2, 0, 4, 1)
(5, 2, 1, 3, 3, 3, 2, 1)


In [3]:
soma = m1 + m2  # Soma vetorial em F_p^n
print(soma)
produto_escalar = 3 * m1 
print(produto_escalar)

(6, 5, 6, 2, 5, 3, 6, 2)
(3, 2, 1, 4, 6, 0, 5, 3)


In [4]:
import binascii

def bytes_to_fp_vetor(byte_string, p, n):
    """Converte uma string de bytes para um vetor em F_p^n"""
    F = GF(p)
    V = VectorSpace(F, n)
    valores = [F(b) for b in bytearray(byte_string)[:n]]
    return V(valores)

# Exemplo: Converter a string "hello!!!" para um vetor em F_7^8
msg_bytes = b"hello!!!"
msg_fp = bytes_to_fp_vetor(msg_bytes, p, n)
print(msg_fp)

(6, 3, 3, 3, 6, 5, 5, 5)


---------------

## Gerador de Bernoulli

O problema LPN (Learning Parity with Noise) baseia-se na dificuldade computacional de resolver sistemas lineares corrompidos por um erro. Especificamente, em LPN, temos um conjunto de equações da forma:
$$ y=Ax+e $$

onde:

- $A$ é uma matriz binária $m×n$ (geralmente aleatória).

- $x$ é um vetor secreto de $n$ bits.

- $e$ é um vetor de erro, onde cada entrada é $1$ com uma pequena probabilidade $\epsilon$ (geralmente pequena, como 0.10.1 ou 0.20.2).

- $y$ é o vetor de observações.

O vetor de erro ee segue uma distribuição Bernoulli com parâmetro ϵϵ, ou seja:
$$ e_i∼Bernoulli(\epsilon)$$

Isso significa que cada bit $e_i$ tem probabilidade $\epsilon$ de ser $1$ (ruído) e $1-\epsilon$ de ser $0$ (sem ruído).

A forma mais direta de implementar um gerador de Bernoulli  $\,\mathcal{B}(\epsilon)$  com a precisão de $\,n\,$ bits, é o algoritmo.
                            $$\mathcal{B}(\epsilon) \;\equiv\;\vartheta \,w\gets \{0,1\}^n\,\centerdot\,\mathsf{if}\,\;\sum_{i=1}^n\,w_i\,2^{-i}\,\leq\, \varepsilon\;\,\mathsf{then}\,\;1\;\,\mathsf{else}\,\;0$$
Aqui  $\,\hat{w} \equiv \sum_{i=1}^n\,2^{-i}\,w_i\,$ é o designado racional de Lebesgue  determinado pela string de bits $\,w\,$.  Em muitos CPU’s ,  $\,\hat{w}\,$ pode ser calculado em tempo constante ; por isso, este é um processo usual para gerar uniformemente racionais no intervalo $\,[0\,,\,1]$.

In [6]:
import random

def bernoulli_lebesgue(epsilon, n=53):
    """
    Gera uma amostra de Bernoulli B(epsilon) usando a construção de Lebesgue.
    - epsilon: parâmetro da distribuição de Bernoulli (0 < epsilon < 1)
    - n: número de bits para a precisão (default: 53, precisão de um double)
    """
    # Gera a string de bits aleatórios {0,1}^n
    w = [random.randint(0, 1) for _ in range(1, n+1)]
    
    # Calcula o racional de Lebesgue
    w_hat = sum(w[i-1] * 2^(-i) for i in range(1,n+1))
    
    return 1 if w_hat <= epsilon else 0

# Teste com epsilon = 0.1
epsilon = 0.1
print([bernoulli_lebesgue(epsilon) for _ in range(10)])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]



O gerador $\,\mathsf{LPN}_{\lambda,\epsilon}(\mathsf{s})\,$, para um segredo $\,\mathsf{s}\gets \mathcal{B}^\lambda\,$,   usa parâmetros $\,\lambda\,$ e $\,\epsilon\,$ definindo-se do seguinte modo


$$\mathsf{LPN}_{\lambda,\epsilon}(\mathsf{s})\;\equiv\quad\vartheta\,{a}\gets\mathcal{B}^\lambda\,\centerdot\,e\gets \mathcal{B}(\varepsilon)\,\centerdot\,\vartheta\,t\gets \mathsf{s}\cdot a + e\,\centerdot\,\langle\,a,t\,\rangle$$ 
        

Aqui  $\,\mathsf{s}\cdot a\,\equiv\,\sum_i\,\mathsf{s}_i\times a_i\;$ denota o produto interno dos dois vetores.

In [7]:
def bernoulli_noise(epsilon):
    """Gerador de Bernoulli B(epsilon) que retorna 1 com probabilidade epsilon e 0 caso contrário."""
    return 1 if random.random() < epsilon else 0

def LPN_generator(lambda_, epsilon):
    """Gera um par (a, t) segundo o protocolo LPN."""
    # Segredo s gerado aleatoriamente com bits {0,1}^λ
    s = vector(GF(2), [randint(0, 1) for _ in range(lambda_)])
    
    # Vetor aleatório a em {0,1}^λ
    a = vector(GF(2), [randint(0, 1) for _ in range(lambda_)])
    
    # Erro e seguindo uma distribuição de Bernoulli B(epsilon)
    e = bernoulli_noise(epsilon)
    
    # Computa t = s ⋅ a + e
    t = s.dot_product(a) + e
    
    return a, t

# Exemplo de uso com lambda = 10 e epsilon = 0.1
lambda_ = 10
epsilon = 0.1
a, t = LPN_generator(lambda_, epsilon)

print("Vetor a:", a)
print("Valor t:", t)

Vetor a: (1, 0, 1, 1, 0, 0, 1, 0, 1, 1)
Valor t: 1


## Operação Choose(b)

Neste protocolo $\,b\in [N]\,$ denota o índice da mensagem que vai ser excluída das transferências legítimas; o criptograma $\,c_b\,$ não pode ser decifrado corretamente pelo receiver porque este agente não conhece uma chave privada que o permita.
    
1. O sender  escolhe o par $\,(\alpha,\ell)\,$ e a função XOF e envia essa informação para o sender;  esta informação determina completamente   a sequência $\;\{\langle\,a_i,u_i\,\rangle\}_{i=1}^\ell\,$  que passa a  formar o “oblivious criterion” ; ambos os agentes podem construir estes elementos.
2. o receiver gera $\,N\,$ segredos $\,\mathsf{s}_k\gets \mathcal{B}^\lambda\,$,  se $k\neq b\,$, e $\,\mathsf{s}_b \gets \bot\,$. Para todo $\,$$$\,k\in [N]\,$ e todo $\,i\in [\ell]\,$ , calcula $\,t_{k,i}$ da seguinte forma
            
$$t_{i,k}\;\gets\;\left\{\begin{array}{lcl}\vartheta\,e_{i,k}\gets \mathcal{B(\epsilon)}\,\centerdot\, a_i\cdot \mathsf{s}_k + e_{i,k} & \text{se} & k\neq b \\ u_i + \sum_{j\neq b}\,t_{i,j} &\text{se}& k=b\end{array}\right.$$

Regista esta ingerar_ruidoformação na sua memória.  

Construímos, para cada $\,i\in[\ell]\,$ , um vetor em $\,\mathcal{B}^N$

$$\mathsf{t}_i\,\equiv\,\{t_{i,k} \;|\;k\in [N]\}\,$$

e envia-os para o sender  como chaves públicas.

c .  o sender   recolhe todas os vetores de chaves públicas $\,\mathsf{t_i}\,$ e verifca as igualdades 

$$\sum_{k\in [N]}\,\mathsf{t}_{i,k}\;=\; u_i$$  $\,$ 

Se, para algum $\,i\in[\ell]\,$  a igualdade não se verifica então termina em falha.
Se se verificar a igualdade então  regista todos os $\,\mathsf{t}_i\,$ na sua memória para transferência futura.

## Operação transfer